# Preprocessing 

In [1]:
import docx
import pandas as pd
import glob
import os
from pathlib import Path
import re
import pickle
import string
import numpy as np
# pip install germalemma

## Loading files into the df and extracting the names and year

In [53]:
# otwieranie wszystkich plikow w folderze i zapisanie ich do df

dir_path = "../data/rel_DEU/"

def get_docx_files(dir_path):
    docx_files = []
    
    for root, _, files in os.walk(dir_path):
        for f in files:
            if f.endswith('.docx'):
                docx_files.append(os.path.join(root, f))
    
    return docx_files

docx_files = get_docx_files(dir_path)

data = []

for file in docx_files:
    # Get the file path
#     file_path = os.path.join(dir_path, file)
    # Open the docx file
    doc = docx.Document(file)
    # Create an empty string to store the text
    text = ''
    # Loop through each paragraph in the document
    for para in doc.paragraphs:
        # Add the text to the string
        text += para.text.strip() + ' '
    # Append the filename and text to the data list
    data.append([file[:-5], text.strip()])

# Create a pandas dataframe from the data
df = pd.DataFrame(data, columns=['f_name', 'raw_txt'])
# df

In [101]:
df.head(2)

,date,year,f_name,raw_txt,txt_1,len_text,cleaned_txt,Denomination
0,2021-06-01,2021-01-01,../data/rel_DEU/Anthroposophen/Anth_Rest/2021-...,*** Date of publication: 06/2021 Link: AGiD Ne...,Kurz nach dem ersten coronabedingten Lockdown ...,1134,Kurz nach dem ersten coronabedingten Lockdown ...,Anthrop
1,2021-04-01,2021-01-01,../data/rel_DEU/Anthroposophen/Anth_Rest/2021-...,*** Date of publication: 04/2021 Link: AGiD Ne...,"In diesem Beitrag beschreibt Christine Rüter, ...",1123,In diesem Beitrag beschreibt Christine Rüter V...,Anthrop


In [104]:
df[df['f_name'].str.contains('chrismon')]

,date,year,f_name,raw_txt,txt_1,len_text,cleaned_txt,Denomination
32,2022-04-01,2022-01-01,../data/rel_DEU/chrismon/chrismon_Rest/2022-04...,*** Date of publication: 27/04/2022 Link: http...,Dagmar Pruin und Düzen Tekkal über Chancenglei...,2439,Dagmar Pruin und Düzen Tekkal über Chancenglei...,Cath_Ch
33,2023-02-01,2023-01-01,../data/rel_DEU/chrismon/chrismon_Rest/2023-02...,*** Date of publication: 27/02/2023 Link: http...,Anna-Nicole Heinrich über besondere Momente un...,621,AnnaNicole Heinrich über besondere Momente und...,Cath_Ch
34,2020-12-01,2020-01-01,../data/rel_DEU/chrismon/chrismon_Rest/2020-12...,*** Date of publication: 14/12/2020 Link: http...,Entwicklungsminister Gerd Müller zum Textilsie...,826,Entwicklungsminister Gerd Müller zum Textilsie...,Cath_Ch
35,2021-08-01,2021-01-01,../data/rel_DEU/chrismon/chrismon_Rest/2021-08...,*** Date of publication: 25/08/2021 Link: http...,Bedford-Strohm über die Flutkatastrophe in Deu...,552,BedfordStrohm über die Flutkatastrophe in Deut...,Cath_Ch
36,2020-12-01,2020-01-01,../data/rel_DEU/chrismon/chrismon_RandomSample...,*** Date of publication: 10/12/2020 Link: http...,Interview: Warum ein Pfarrer die Corona-Maßnah...,1265,Interview Warum ein Pfarrer die CoronaMaßnahme...,Cath_Ch
...,...,...,...,...,...,...,...,...
101,2021-05-01,2021-01-01,../data/rel_DEU/chrismon/chrismon_RandomSample...,*** Date of publication: 03/05/2021 Link: http...,"Bernd Osterloh, Personalvorstand und ehemalige...",726,Bernd Osterloh Personalvorstand und ehemaliger...,Cath_Ch
102,2020-04-01,2020-01-01,../data/rel_DEU/chrismon/chrismon_RandomSample...,*** Date of publication: 22/04/2020 Link: http...,Kirche lebt durch Gemeinschaft. In Zeiten von ...,195,Kirche lebt durch Gemeinschaft In Zeiten von C...,Cath_Ch
103,2020-11-01,2020-01-01,../data/rel_DEU/chrismon/chrismon_RandomSample...,*** Date of publication: 02/11/2020 Link: http...,Angst vor Krankheit und Einsamkeit: Corona und...,2581,Angst vor Krankheit und Einsamkeit Corona und ...,Cath_Ch
104,2021-01-01,2021-01-01,../data/rel_DEU/chrismon/chrismon_RandomSample...,*** Date of publication: 28/01/2021 Link: http...,"""Spielraum! Sieben Wochen ohne Blockaden"" - Fa...",1239,Spielraum Sieben Wochen ohne Blockaden Fastena...,Cath_Ch


In [56]:
df['date']=df['f_name'].apply(lambda x: x.split('/')[5].split('_')[0])
df['date']=df['date'].str[:7]
df['year']=df['date'].str[:4]

In [60]:
df['date']=np.where(df['date']=='2020-Is','2020-01',df['date'])
df['date']=np.where(df['date']=='2021-Is','2021-01',df['date'])
df['date']=np.where(df['date']=='2022-Is','2022-01',df['date'])

In [61]:
df['date'] = df['date'].replace('0000-00-00', np.nan)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m')

In [62]:
df['year'] = pd.to_datetime(df['year'], format='%Y', errors='coerce')

In [63]:
# zmiana kolejności, aby źródło było po roku
df = df.iloc[:, [2,3,0,1]].copy()

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797 entries, 0 to 796
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     797 non-null    datetime64[ns]
 1   year     797 non-null    datetime64[ns]
 2   f_name   797 non-null    object        
 3   raw_txt  797 non-null    object        
dtypes: datetime64[ns](2), object(2)
memory usage: 25.0+ KB


In [73]:
# Function to extract text after "Text of Article:"
def extract_article_text(text):
    article_marker = "Text of Article:"
    article_index = text.find(article_marker)
    if article_index != -1:
        return text[article_index + len(article_marker):].strip()
    else:
        return text

# Apply the function to create the new column
df['txt_1'] = df['raw_txt'].apply(extract_article_text)

In [75]:
df[df['txt_1'].isna()]

,date,year,f_name,raw_txt,txt_1


## text cleaning

In [77]:
def clean(text):
    text = re.sub("http\S+|www\S+|fb.com\S+|programm.ard.de\S+", '', text)
    text = ''.join([char for char in text if char not in '–!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~“”·‘'])
    text = re.sub('\n',' ',text)
    text = re.sub('‹|›|„','',text)
    text = re.sub(r'\b\d+\b\s*$', '', text)
    text = re.sub(r'\b\w*\d+\w*\b', '', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'•\t', '', text)
    return text.strip()

In [78]:
df['cleaned_txt'] = df['txt_1'].apply(clean)

In [ ]:
df['len_text'] = df['txt_1'].apply(lambda x: len(x.split()))

In [84]:
# sprawdzam jak długi jest tekst i czy czegoś nie ucięło, wyrywkowo
df['len_text'].describe()

count      797.000000
mean       706.422836
std       1196.333611
min        120.000000
25%        320.000000
50%        446.000000
75%        778.000000
max      29425.000000
Name: len_text, dtype: float64

In [85]:
min_row = df.loc[df['len_text'].idxmin()]

# Print the row
print(min_row)

date                                         2021-07-01 00:00:00
year                                         2021-01-01 00:00:00
f_name         ../data/rel_DEU/IslamischeZeitung/IZ_RandomSam...
raw_txt        *** Date of publication: 17/07/2021 Link: http...
txt_1          Jakarta (KNA). Indonesien hat in dieser Woche ...
len_text                                                     120
cleaned_txt    Jakarta KNA Indonesien hat in dieser Woche Ind...
Name: 106, dtype: object


## Klasyfikacja do denominacji

In [94]:
dict = {
        "Bistum" : 'Cath_Ch', "BR" : 'Cath_Ch', "DBK" : 'Cath_Ch', 'chrismon': 'Cath_Ch', 'katholisch.de': 'Cath_Ch',
        
        "ELKB" : 'Protest', "EKD" : 'Protest', "EVLKS" : 'Protest',

        "AGiD" : 'Anthrop', "Ant" : 'Anthrop',

        "IRD" : 'Muslim', "KRM" : 'Muslim', "ZDM" : 'Muslim', "DITIB" : 'Muslim', "VIKZ" : 'Muslim', "IRD" : 'Muslim', "ZMD" : 'Muslim', "Islamrat": 'Muslim', "IslamischeZeitung": "IRD"
    }

        

In [95]:
for key, value in dict.items():
    df.loc[df['f_name'].str.contains(key), 'Denomination'] = value


In [96]:
df[df['Denomination'].isna()]

,date,year,f_name,raw_txt,txt_1,len_text,cleaned_txt,Denomination


In [97]:
df

,date,year,f_name,raw_txt,txt_1,len_text,cleaned_txt,Denomination
0,2021-06-01,2021-01-01,../data/rel_DEU/Anthroposophen/Anth_Rest/2021-...,*** Date of publication: 06/2021 Link: AGiD Ne...,Kurz nach dem ersten coronabedingten Lockdown ...,1134,Kurz nach dem ersten coronabedingten Lockdown ...,Anthrop
1,2021-04-01,2021-01-01,../data/rel_DEU/Anthroposophen/Anth_Rest/2021-...,*** Date of publication: 04/2021 Link: AGiD Ne...,"In diesem Beitrag beschreibt Christine Rüter, ...",1123,In diesem Beitrag beschreibt Christine Rüter V...,Anthrop
2,2020-05-01,2020-01-01,../data/rel_DEU/Anthroposophen/Anth_randomSamp...,*** Date of publication: 05/2020 Link: AGiD Ne...,Das Arbeitskollegium der Anthroposophischen Ge...,892,Das Arbeitskollegium der Anthroposophischen Ge...,Anthrop
3,2021-01-01,2021-01-01,../data/rel_DEU/Anthroposophen/Anth_randomSamp...,*** Date of publication: 01/2021 Link: AGiD Ne...,Ich erinnere mich noch gut an den Jahresbeginn...,849,Ich erinnere mich noch gut an den Jahresbeginn...,Anthrop
4,2021-05-01,2021-01-01,../data/rel_DEU/Anthroposophen/Anth_randomSamp...,*** Date of publication: 05/2021 Link: AGiD Ne...,Das Signum der Corona-Zeit kann als Weckruf ve...,307,Das Signum der CoronaZeit kann als Weckruf ver...,Anthrop
...,...,...,...,...,...,...,...,...
792,2020-12-01,2020-01-01,../data/rel_DEU/katholisch.de/katholischDe_Res...,Laschet stellt öffentliche Weihnachtsgottesdie...,Laschet stellt öffentliche Weihnachtsgottesdie...,255,Laschet stellt öffentliche Weihnachtsgottesdie...,Cath_Ch
793,2020-04-01,2020-01-01,../data/rel_DEU/katholisch.de/katholischDe_Res...,"Weiterhin ""würdig""? Kommunionspendung außerhal...","Weiterhin ""würdig""? Kommunionspendung außerhal...",728,Weiterhin würdig Kommunionspendung außerhalb d...,Cath_Ch
794,2020-03-01,2020-01-01,../data/rel_DEU/katholisch.de/katholischDe_Res...,Bischofskonferenz veröffentlicht Karfreitagsfü...,Bischofskonferenz veröffentlicht Karfreitagsfü...,458,Bischofskonferenz veröffentlicht Karfreitagsfü...,Cath_Ch
795,2020-08-01,2020-01-01,../data/rel_DEU/katholisch.de/katholischDe_Res...,Kirchliches Leben ist mehr als die Feier des G...,Kirchliches Leben ist mehr als die Feier des G...,326,Kirchliches Leben ist mehr als die Feier des G...,Cath_Ch


In [99]:
df.to_csv('../data/clean/rel_deu_clean.csv')

In [100]:
# from germalemma import GermaLemma

# lemmatizer = GermaLemma()

# # passing the word and the POS tag ("N" for noun)
# lemma = lemmatizer.find_lemma('Feinstaubbelastungen', 'N')
# print(lemma)
# # -> lemma is "Feinstaubbelastung"